# Main Analysis File
This file contains the main data analysis code for the project. It is divided into 3 main parts:
1. Data Cleaning
2. Exploratory Data Analysis
3. Machine Learning

In [5]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Data Cleaning

In [6]:
# filepaths of all of the csv data files to be analyzed
autonest_csv = "CEIP_csv/AutoNest.csv"
autonest_strategy_csv = "CEIP_csv/AutoNestStrategy.csv"
material_csv = "CEIP_csv/Material.csv"
nest_csv = "CEIP_csv/Nest.csv"
part_csv = "CEIP_csv/Part.csv"
performance_csv = "CEIP_csv/Performance.csv"

# read in all of these csv files as pandas dataframes
# autonest_df = pd.read_csv(autonest_csv)
# autonest_strategy_df = pd.read_csv(autonest_strategy_csv)
# material_df = pd.read_csv(material_csv)
# nest_df = pd.read_csv(nest_csv)
# part_df = pd.read_csv(part_csv)
# performance_df = pd.read_csv(performance_csv)

# change the working directory to the root directory of this project
os.chdir("..")

# read in the most relevant .csv 
part_df = pd.read_csv(part_csv)


In [7]:
# count the number of rows in the part_df
part_df.shape[0] - # 30970284 - about 30 million rows 

# show the first 5 rows of the part_df
print(part_df.head())

# print summary statistics about the part_df 
print(part_df.describe())

30970284
   ixPart  ixJobSummary  dLength   dWidth      dArea  cRequired  cNested  \
0       1             2  34.2210  34.0000  1126.3525          1        1   
1       2             2  12.6457  12.6457   119.0266          1        1   
2       3             2  24.6766  11.8546    95.4638          1        1   
3       4             3  31.2500  22.5000   600.0000          1        1   
4       5             4  34.2210  34.0000  1126.3525          1        1   

   ixMaterial  fExtShape   dExtArea  dExtBoundaryDist  dExtContainedDist  \
0           2          0  1148.0000           16.2732            45.7153   
1           2          0   119.0266            4.9484            13.0959   
2           2          0    99.7099            2.8510            17.7563   
3           2          5   600.0000            9.6000            30.3000   
4           2          0  1148.0000           16.2732            45.7153   

   dLgIntArea  dLgIntBoundaryDist  dLgIntContainedDist  dLgExtConArea  \
0   

KeyboardInterrupt: 

## Exploratory Data Analysis

## Machine Learning